# Irisの分類

In [1]:
# coding: UTF-8

import chainer
from chainer import Variable, Chain, optimizers
import chainer.links as L
import chainer.functions as F

import numpy as np
from sklearn import datasets # Scikit Learn にIris dataset が含まれている

In [2]:
# Iris データの読み込み
iris_data = datasets.load_iris()

In [3]:
x = iris_data.data.astype(np.float32)
t = iris_data.target
n = t.size

In [4]:
# 教師データの下処理
t_matrix = np.zeros(3 * n).reshape(n, 3).astype(np.float32)
for i in range(n):
    t_matrix[i, t[i]] = 1.0

In [5]:
# 訓練用データとテスト用データ 半分が訓練用データで残りがテスト用データ
indexes = np.arange(n)
indexes_train = indexes[indexes%2 != 0]
indexes_test = indexes[indexes%2 == 0]

x_train = x[indexes_train, :] # 訓練用 入力
t_train = t_matrix[indexes_train, :] # 訓練用 正解
x_test = x[indexes_test, :] # テスト用 入力
t_test = t[indexes_test] # テスト用 正解

x_train_v = Variable(x_train)
t_train_v = Variable(t_train)
x_test_v = Variable(x_test)

In [6]:
# Chain の記述
class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1 = L.Linear(4, 6),
            l2 = L.Linear(6, 6),
            l3 = L.Linear(6, 3),
        )
        
    def predict(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = F.sigmoid(self.l2(h1))
        h3 = self.l3(h2)
        return h3

In [7]:
# モデルとOptimizerの設定
model = IrisChain()
optimizer = optimizers.Adam()
optimizer.setup(model)

In [8]:
# 学習
for i in range(10000):
    
    model.cleargrads()
    y_train_v = model.predict(x_train_v)
    
    # 損失関数による誤差の計算、この場合は平均2乗誤差
    loss = F.mean_squared_error(y_train_v, t_train_v)
    loss.backward()
    
    # Optimizer による重みの更新
    optimizer.update()

In [9]:
# テスト
model.cleargrads()
y_test_v = model.predict(x_test_v)
y_test = y_test_v.data

In [11]:
# 正解数のカウント
correct = 0
rowCount = y_test.shape[0]
for i in range(rowCount):
    maxIndex = np.argmax(y_test[i, :])
    print(y_test[i, :], maxIndex)
    if maxIndex == t_test[i]:
        correct += 1

# 正解率
print("Correct:" , correct, "Total:", rowCount, "Accuracy:", correct / rowCount * 100, "%")

[  1.00166655e+00  -4.57245857e-04  -3.74421477e-03] 0
[  1.00120521e+00  -4.19486314e-04  -2.28126347e-03] 0
[  1.00192893e+00  -6.91849738e-04  -3.68157029e-03] 0
[  1.00167131e+00  -1.37624517e-03   6.66245818e-04] 0
[ 0.99765593  0.00156742  0.00206603] 0
[  1.00174069e+00  -3.57512385e-04  -4.40019369e-03] 0
[  9.99650657e-01   9.55235213e-04  -2.57307291e-03] 0
[ 0.99998659  0.00255872 -0.00970215] 0
[ 1.00288391 -0.00148458 -0.00353172] 0
[ 1.00216615 -0.00128808 -0.00164597] 0
[  1.00017202e+00   1.57218426e-04  -7.25522637e-04] 0
[  1.00148606e+00   3.78068537e-04  -6.53192401e-03] 0
[ 0.99491054  0.00312754  0.00578673] 0
[  9.99629617e-01  -2.55633146e-04   4.02507186e-03] 0
[  1.00137806e+00  -1.99604779e-04  -3.79863381e-03] 0
[ 0.9976002   0.00163109  0.00224061] 0
[  1.00142288e+00   4.55360860e-04  -6.60683215e-03] 0
[  9.99588370e-01   8.97746533e-04  -2.06997991e-03] 0
[  1.00105929e+00   7.10409135e-04  -6.37997687e-03] 0
[  9.99989748e-01   9.90442932e-05  -1.135617